# 라이브러리 호출

In [31]:
!pip install selenium
!pip install stylecloud
!pip install youtube_transcript_api
!pip install tqdm

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
## 댓글 & 제목/URL
import matplotlib.pyplot as plt
from konlpy.tag import Hannanum
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
from collections import Counter
from wordcloud import WordCloud
# Expected chromedriver.exe

import tqdm
import random

In [2]:
## 자막
from youtube_transcript_api import YouTubeTranscriptApi

# 함수 정의

In [3]:
# 이모티콘 제거
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

# 분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,"\n\r#\ufeff\u200d]')

In [4]:
def get_comment(url, num_scroll = 50):
    #num_scroll = int(input('input num of scroll (50 recommend) : '))
    browser = Chrome()
    browser.get(url)
    browser.implicitly_wait(2)
    body = browser.find_element_by_tag_name('body')
    yt_id = []
    yt_content = []
    yt_likes = []
    while num_scroll:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        num_scroll -= 1
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
    browser.close()
    result = html.find_all("div", id='main')
    for i in result:
        i = BeautifulSoup(str(i), 'html.parser')
        try:
            yt_id.append(i.find('a', id='author-text').text)
            yt_content.append(i.find('div', id='comment-content').text)
            yt_likes.append(i.find('span', id='vote-count-middle').text)
        except:
            print('Exception!!!!!!!!!')
    df = pd.DataFrame({'id': yt_id, 'content': yt_content, 'likes': yt_likes})
    df.index = df.index.map(str)
    
    return df

def preprocess_text(text):
    text = emoji_pattern.sub('', text)
    text = han.sub('', text)
    return text

def preprocess_likes(like):
    number_like = ''.join(filter(str.isdigit, like))
    return int(number_like)

def get_id(url):
    video_id = re.search(r"v=([A-Za-z0-9_-]+)", url).group(1)
    return video_id

def comment_cleaner(text):
    return re.sub(r'간략히자세히 보기', '', text)

In [21]:
df.head()

,id,content,likes
0,\n@user-uf7in4sn5z\n,1 조언해주는 사람을 신중하게 고르기2 나에게 맞는 말들을 잘 선별하기3 책임을 ...,\n 153\n
1,\n@user-vvsvv\n,조언을 해 줄 사람을 신중하게 구한다는 말.. 정말 그게 중요한 것 같아요간략히자...,\n 121\n
2,\n@kkm9637\n,내 삶에 대해서 판단력을 가지는 것과주체적으로 생각하는 근육 키우는 게정말 중요한...,\n 7\n
3,\n@user-rd9jc7by4v\n,딱 저를 위한 영상... 제 감정이나 의견을 존중하지 않는 부모님 아래에서 자라기...,\n 37\n
4,\n@nemblis\n,이 영상은 정말 많은 사람들의 생각과 감정을 대변하는 것 같아요. 제 자신에 대한...,\n 3\n


# 제목 및 URL 

In [286]:
url = 'https://www.youtube.com/@TheoBaker./videos'

In [287]:
browser = Chrome()
browser.get(url)
browser.maximize_window()

In [288]:
last_height = browser.execute_script("return document.getElementById('content').scrollHeight") 
while True : 
    body = browser.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.END)
    time.sleep(2.0)
    current_height = browser.execute_script("return document.getElementById('content').scrollHeight")
    if current_height == last_height :
        break 
        
    last_height = current_height
    print(last_height,current_height)
    if current_height > 20000:
        break
video_links = browser.find_elements(By.ID,'video-title-link')

4452 4452
6254 6254
8314 8314
10116 10116
12175 12175
13977 13977
16037 16037
17839 17839
19899 19899
21681 21681


In [289]:
links = []
titles = []
for i in range(len(video_links)):
    if video_links[i].text:
        links.append(video_links[i].get_attribute('href'))
        titles.append(video_links[i].text)
    else:
        break
browser.close()

In [290]:
data = pd.DataFrame({'title': titles, 'link': links} )

In [291]:
data['video_num'] = data['link'].apply(get_id)
print(len(data))
data = data.sample(frac=1).reset_index(drop=True)

328


,title,link,video_num
0,YOUTUBER BAKE OFF vs W2S,https://www.youtube.com/watch?v=QVpd-_RdWss,QVpd-_RdWss
1,Mikel Arteta Became Our Coach for a Day!,https://www.youtube.com/watch?v=FFpyDoBwjSk,FFpyDoBwjSk
2,I Got INJURED at The Sidemen Charity Match!,https://www.youtube.com/watch?v=l629KfWCwOE,l629KfWCwOE
3,How Good is Alisha Lehmann at Football?,https://www.youtube.com/watch?v=ABrNryPZ9P8,ABrNryPZ9P8
4,England vs Scotland YouTube Football Match - T...,https://www.youtube.com/watch?v=4sDSorCRHeg,4sDSorCRHeg


In [293]:
data

,title,link,video_num
0,LOOK WHO CAME TO VISIT!,https://www.youtube.com/watch?v=m20vqgSq0Wg,m20vqgSq0Wg
1,FOOTBALL ON CRUTCHES!!,https://www.youtube.com/watch?v=H24L_Uu2K9s,H24L_Uu2K9s
2,DAVID BECKHAM VS ZIDANE FOOTBALL CHALLENGE,https://www.youtube.com/watch?v=DIjWCtnYmUw,DIjWCtnYmUw
3,SCORING A HAT-TRICK ON HASHTAG UNITED DEBUT!?,https://www.youtube.com/watch?v=O4gSjFQsN4M,O4gSjFQsN4M
4,I SCORED GOAL OF THE SEASON!!,https://www.youtube.com/watch?v=BTcPFqsAngk,BTcPFqsAngk
...,...,...,...
323,ATTRIBUTE CHALLENGE VS CHRISTIAN FUCHS,https://www.youtube.com/watch?v=O5yooLVwMb8,O5yooLVwMb8
324,PLAYER OF THE YEAR KANTE FOOTBALL CHALLENGE!! ...,https://www.youtube.com/watch?v=GXV1lrkQpCk,GXV1lrkQpCk
325,DAVID BECKHAM PASSING CHALLENGE | FOOTBALL IN ...,https://www.youtube.com/watch?v=L91fqbHTb8Q,L91fqbHTb8Q
326,FOOTBALL TRAINING IN FRANCE!,https://www.youtube.com/watch?v=wwtL5Ym3GhY,wwtL5Ym3GhY


# 첫 영상관한 데이터 생성하기

In [294]:
srt = YouTubeTranscriptApi.get_transcript(data.iloc[0,2], languages=['en'])
text = ''
for j in range(len(srt)):
    text += srt[j]['text'] + ''
df = get_comment(data.iloc[0,1],15)

In [295]:
df['content'] = df['content'].apply(preprocess_text)
df['content'] = df['content'].apply(comment_cleaner)
df['likes'] = df['likes'].apply(preprocess_likes)

In [296]:
df = df.sort_values('likes', ascending = False)
#df = df[df['content'].apply(lambda x: len(str(x)) > 20)].reset_index(drop = True)

In [297]:
most_like = df.iloc[0,1]
middle_like = df.iloc[1,1]
fewest_like = df.iloc[2,1]


#while True:
 #   middle_like = df.iloc[random.randint(1, len(df) - 2), 1]
  #  if len(middle_like) > 30:
   #     break


df_partial = pd.DataFrame({'title':[data.iloc[0,0], data.iloc[0,0], data.iloc[0,0]],
                           'subtitle': [text, text, text],
                          'comment': [most_like, middle_like, fewest_like]})

In [298]:
df_partial

,title,subtitle,comment
0,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Be honest now guys does anyone skip to the en...
1,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Who else knew it was Chris just from the boots
2,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Theo I hate school I hate work But I don't ha...


# 나머지 영상 크롤링후 concat

In [300]:
for i in tqdm.tqdm(range(13,52)):
    #자막 추출
    srt = YouTubeTranscriptApi.get_transcript(data.iloc[i,2], languages=['en'])
    text = ''
    for j in range(len(srt)):
        text += srt[j]['text'] + ''
    # 댓글 추출
    df = get_comment(data.iloc[i,1],10)
    df['content'] = df['content'].apply(preprocess_text)
    df['content'] = df['content'].apply(comment_cleaner)
    df['likes'] = df['likes'].apply(preprocess_likes)
    
    df = df.sort_values('likes', ascending = False)
    df = df[df['content'].apply(lambda x: len(str(x)) > 20)].reset_index(drop = True)
    
    most_like = df.iloc[0,1]
    middle_like = df.iloc[random.randint(2,len(df)-2),1]
    fewest_like = df.iloc[-1,1]
    
    
    print(len(most_like), len(middle_like), len(fewest_like))
    
        
        
    df_partial_new = pd.DataFrame({'title':[data.iloc[i,0], data.iloc[i,0], data.iloc[i,0]],
                                   'subtitle': [text, text, text],
                          'comment': [most_like, middle_like, fewest_like]})
    df_partial = pd.concat([df_partial, df_partial_new])
    if len(df_partial) == 150:
        break

  3%|██▏                                                                                | 1/39 [00:25<15:51, 25.05s/it]

122 53 100


  5%|████▎                                                                              | 2/39 [00:47<14:21, 23.28s/it]

139 65 303


  8%|██████▍                                                                            | 3/39 [01:11<14:15, 23.77s/it]

...

40 40 41


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [12:46<00:16, 16.11s/it]

38 31 21


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [13:01<00:20, 20.57s/it]

46 70 80


In [303]:
df_partial

,title,subtitle,comment
0,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Be honest now guys does anyone skip to the en...
1,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Who else knew it was Chris just from the boots
2,LOOK WHO CAME TO VISIT!,so what's going on guys welcome back toanother...,Theo I hate school I hate work But I don't ha...
0,FOOTBALL ON CRUTCHES!!,boys and girls before you start thisvideo I ha...,I live in australia but watching your videos ...
1,FOOTBALL ON CRUTCHES!!,boys and girls before you start thisvideo I ha...,MAKE YOUR FIFA RATING 78 AND 9 PLEASE I LOVE ...
...,...,...,...
1,TOTY VOLLEY DRAFTWARS VS DANIEL CUTTING!!,cool what's going on guys welcome backto anoth...,It’s all about that chemistry
2,TOTY VOLLEY DRAFTWARS VS DANIEL CUTTING!!,cool what's going on guys welcome backto anoth...,Great as always theo
0,ULTIMATE FREEKICK CHALLENGE | THEO VS CHARLIE #3,what's going on guys welcome back to thethird ...,What should the next challenge be Be creative
1,ULTIMATE FREEKICK CHALLENGE | THEO VS CHARLIE #3,what's going on guys welcome back to thethird ...,Do a mikitarian and giroud volly challenge bu...


In [304]:
df_partial.to_excel('NLP_theobaker.xlsx', index=False)